In [1]:
!pip install pandas
!pip install music21


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [29]:
import pickle
import pandas as pd
import music21
from datetime import datetime, timedelta

In [30]:
piece_id_counter = 1
phrase_id_counter = 1

In [31]:
data = []

files = ["data_alsace"]
for file in files:
    with open('../data/data/{}.pkl'.format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

In [32]:
df = pd.DataFrame(columns=['PhraseID', 'Note', 'TimeStamp', 'PieceID'])
piece_id_counter = 1
phrase_id_counter = 1
previous_note = None

for piece in data:
    piece_id = "PIECE{:05}".format(piece_id_counter)
    timestamp = datetime.now()

    for i in range(len(piece[0])):
        if piece[0][i] == 128:
            note = "rest"
            if previous_note is not None:
                distance = piece[0][i-1] - previous_note
            else:
                distance = 0
        else:
            current_note = music21.note.Note(piece[0][i]).pitch.midi
            if previous_note is not None:
                distance = current_note - previous_note
            else:
                distance = 0
            note = distance
            previous_note = current_note
        
        duration = piece[1][i]
        phrase_id = "PHRASE{:06}".format(phrase_id_counter)
        timestamp += timedelta(seconds=10 * int(duration))

        new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': [note], 'TimeStamp': [timestamp.strftime("%d/%m/%y %H:%M:%S")], "PieceID": [piece_id]})
        df = pd.concat([df, new_item], ignore_index=True)
        
        if piece[2][i] == 1:
            timestamp = datetime.now()
            phrase_id_counter += 1
            new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': 'END', 'TimeStamp': [timestamp.strftime("%d/%m/%y %H:%M:%S")], "PieceID": [piece_id]})
            df = pd.concat([df, new_item], ignore_index=True)
        
    piece_id_counter += 1
    
    if piece_id_counter % 1000 == 0:
        df.to_csv("log_data{}.csv".format(piece_id_counter))
        df = pd.DataFrame(columns=['PhraseID', 'Note', 'TimeStamp', 'PieceID'])
   

In [33]:
df.to_csv("log_data.csv")